<a href="https://colab.research.google.com/github/vartikagpt10/EEG-based-ERP-Detection-for-practical-BCI/blob/main/P300_Classification_using_Pyriemann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download and Extract Data

In [ ]:
!pip install --upgrade scikit-learn
!pip install --upgrade gdown pyriemann tensorflow imbalanced-learn xgboost
!gdown https://drive.google.com/uc?id=16x3lxR6KBc4OQv83voXnE88J0KzbeLFx -O /tmp/Data.zip

import zipfile
local_zip = '//tmp/Data.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp/Data')
zip_ref.close()

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.23.2)
Requirement already up-to-date: gdown in /usr/local/lib/python3.6/dist-packages (3.12.2)
Requirement already up-to-date: pyriemann in /usr/local/lib/python3.6/dist-packages (0.2.6)
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.3.1)
Requirement already up-to-date: imbalanced-learn in /usr/local/lib/python3.6/dist-packages (0.7.0)
Requirement already up-to-date: xgboost in /usr/local/lib/python3.6/dist-packages (1.2.1)
Downloading...
From: https://drive.google.com/uc?id=16x3lxR6KBc4OQv83voXnE88J0KzbeLFx
To: /tmp/Data.zip
301MB [00:01, 292MB/s]


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
import os
from scipy.signal import filtfilt, cheby2
import matplotlib.pyplot as plt

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

In [ ]:
train_folder_path = '/tmp/Data/Training set/'
validation_folder_path = '/tmp/Data/Validation set/'
test_folder_path = '/tmp/Data/Test set/'

# Import Data

In [ ]:
c=0
x=os.listdir(train_folder_path)
x.sort()
print(x)
for i in x:
  c+=1
  if c==1:
    temp=sio.loadmat(train_folder_path+i)
    dat=np.squeeze(temp['epo_tr']['x']).tolist()
    dat=np.array(dat)
    lab=np.squeeze(temp['epo_tr']['y']).tolist()
    lab=np.array(lab)[0,:]
    training_data=dat
    training_labels=lab
    continue
  temp=sio.loadmat(train_folder_path+i)
  dat=np.squeeze(temp['epo_tr']['x']).tolist()
  dat=np.array(dat)
  lab=np.squeeze(temp['epo_tr']['y']).tolist()
  lab=np.array(lab)[0,:]
  training_data=np.concatenate((training_data,dat),axis=2)
  training_labels=np.concatenate((training_labels,lab))
training_data=training_data.reshape((2700,56,100))
training_data=training_data[:,:32,:]
print("Training data shape:",training_data.shape)
print("Training labels shape:",training_labels.shape)

['Data_Sample01.mat', 'Data_Sample02.mat', 'Data_Sample03.mat', 'Data_Sample04.mat', 'Data_Sample05.mat', 'Data_Sample06.mat', 'Data_Sample07.mat', 'Data_Sample08.mat', 'Data_Sample09.mat', 'Data_Sample10.mat', 'Data_Sample11.mat', 'Data_Sample12.mat', 'Data_Sample13.mat', 'Data_Sample14.mat', 'Data_Sample15.mat']
Training data shape: (2700, 32, 100)
Training labels shape: (2700,)


In [ ]:
c=0
x=os.listdir(validation_folder_path)
x.sort()
print(x)
for i in x:
  c+=1
  if c==1:
    temp=sio.loadmat(validation_folder_path+i)
    dat=np.squeeze(temp['epo_val']['x']).tolist()
    dat=np.array(dat)
    lab=np.squeeze(temp['epo_val']['y']).tolist()
    lab=np.array(lab)[0,:]
    validation_data=dat
    validation_labels=lab
    continue
  temp=sio.loadmat(validation_folder_path+i)
  dat=np.squeeze(temp['epo_val']['x']).tolist()
  dat=np.array(dat)
  lab=np.squeeze(temp['epo_val']['y']).tolist()
  lab=np.array(lab)[0,:]
  validation_data=np.concatenate((validation_data,dat),axis=2)
  validation_labels=np.concatenate((validation_labels,lab))
validation_data=validation_data.reshape((900,56,100))
validation_data=validation_data[:,:32,:]
print("Validation data shape:",validation_data.shape)
print("Validation labels shape:",validation_labels.shape)

['Data_Sample01.mat', 'Data_Sample02.mat', 'Data_Sample03.mat', 'Data_Sample04.mat', 'Data_Sample05.mat', 'Data_Sample06.mat', 'Data_Sample07.mat', 'Data_Sample08.mat', 'Data_Sample09.mat', 'Data_Sample10.mat', 'Data_Sample11.mat', 'Data_Sample12.mat', 'Data_Sample13.mat', 'Data_Sample14.mat', 'Data_Sample15.mat']
Validation data shape: (900, 32, 100)
Validation labels shape: (900,)


In [ ]:
c=0
x=os.listdir(test_folder_path)
x.sort()
print(x)
for i in x:
  c+=1
  if c==1:
    temp=sio.loadmat(test_folder_path+i)
    dat=np.squeeze(temp['epo_te']['x']).tolist()
    dat=np.array(dat)

    test_data=dat
    test_labels=lab
    continue
  temp=sio.loadmat(test_folder_path+i)
  dat=np.squeeze(temp['epo_te']['x']).tolist()
  dat=np.array(dat)

  test_data=np.concatenate((test_data,dat),axis=2)

test_data=test_data.reshape((900,56,100))
test_data=test_data[:,:32,:]
print("Test data shape:",test_data.shape)


['Data_Sample01.mat', 'Data_Sample02.mat', 'Data_Sample03.mat', 'Data_Sample04.mat', 'Data_Sample05.mat', 'Data_Sample06.mat', 'Data_Sample07.mat', 'Data_Sample08.mat', 'Data_Sample09.mat', 'Data_Sample10.mat', 'Data_Sample11.mat', 'Data_Sample12.mat', 'Data_Sample13.mat', 'Data_Sample14.mat', 'Data_Sample15.mat']
Test data shape: (900, 32, 100)


In [ ]:
# dat=np.squeeze(temp['epo_val']['y']).tolist()
# dat=np.array(dat)[0,:]
# dat

# Filtering

In [ ]:
def chebyBandpass(signal, lowcut, fs, order):
  nyq=0.5*fs
  low = lowcut/nyq
  # high = highcut / nyq
  b, a = cheby2(order,12, low, btype='low')
  filtered = filtfilt(b, a, signal)
  return filtered

def filterall(data):
  filtered=[]
  for trial in data:
    temp_filtered=[]
    for channel in trial:
      temp_filtered.append(chebyBandpass(channel,20,100,3))
    filtered.append(temp_filtered)
  return np.array(filtered)

training_data=filterall(training_data)
validation_data=filterall(validation_data)
test_data=filterall(test_data)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

from pyriemann.estimation import ERPCovariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.classification import TSclassifier

from sklearn.metrics import classification_report
import sklearn.metrics as metrics
from sklearn import svm
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier

from imblearn.combine import SMOTETomek

In [ ]:
training_labels_all=training_labels
validation_labels_all=validation_labels
auc1=[]
auc2=[]
auc3=[]
proba1=[]
proba2=[]
proba3=[]
val_proba1=[]
val_proba2=[]
val_proba3=[]
score1=[]
score2=[]
score3=[]
for sub in range(15):
  filt_train=training_data[(sub)*180:(sub+1)*180,:,:]
  filt_val=validation_data[(sub)*60:(sub+1)*60,:,:]
  filt_test=test_data[(sub)*60:(sub+1)*60,:,:]
  training_labels=training_labels_all[(sub)*180:(sub+1)*180]
  validation_labels=validation_labels_all[(sub)*60:(sub+1)*60]

  print(filt_train.shape)
  print(filt_val.shape)
  print(filt_test.shape)

  covmat = ERPCovariances(estimator='lwf').fit(filt_train, training_labels)
  cov_train = covmat.transform(filt_train)
  ts_train = TangentSpace().fit_transform(cov_train)

  cov_val = covmat.transform(filt_val)
  ts_val = TangentSpace().fit_transform(cov_val)

  cov_test = covmat.transform(filt_test)
  ts_test = TangentSpace().fit_transform(cov_test)

  print(ts_train.shape)
  print(training_labels.shape)
  print(ts_val.shape)
  print(validation_labels.shape)
  print(ts_test.shape)
  print(cov_train.shape)
  print(cov_val.shape)
  print(cov_test.shape)

  training_labels_orig=training_labels
  oversample=SMOTETomek()
  ts_train, training_labels = oversample.fit_resample(ts_train,training_labels)
######## MODEL 1 SVM ########
  model = svm.SVC(probability=True)
  model.fit(ts_train, training_labels)

  training_predicted= model.predict(ts_train)
  validation_pred=model.predict(ts_val)
  score= validation_pred==validation_labels
  score1.append(score.mean())
  probs = model.predict_proba(ts_val)
  val_proba1.append(probs)
  preds = probs[:,1]
  fpr, tpr, threshold = metrics.roc_curve(validation_labels, preds)
  roc_auc = metrics.auc(fpr, tpr)
  auc1.append(roc_auc)
  proba1.append(model.predict_proba(ts_test))
######## MODEL 2 xgboost ########
  model = XGBClassifier(probability=True)
  model.fit(ts_train, training_labels)

  training_predicted= model.predict(ts_train)
  validation_pred=model.predict(ts_val)
  score= validation_pred==validation_labels
  score2.append(score.mean())
  probs = model.predict_proba(ts_val)
  val_proba2.append(probs)
  preds = probs[:,1]
  fpr, tpr, threshold = metrics.roc_curve(validation_labels, preds)
  roc_auc = metrics.auc(fpr, tpr)
  auc2.append(roc_auc)
  proba2.append(model.predict_proba(ts_test))
######## MODEL 3 tsclassifier ########
  training_labels=training_labels_orig
  model = TSclassifier(clf=XGBClassifier())
  model.fit(cov_train, training_labels)

  training_predicted= model.predict(cov_train)
  validation_pred=model.predict(cov_val)
  score= validation_pred==validation_labels
  score3.append(score.mean())
  probs = model.predict_proba(cov_val)
  val_proba3.append(probs)
  preds = probs[:,1]
  fpr, tpr, threshold = metrics.roc_curve(validation_labels, preds)
  roc_auc = metrics.auc(fpr, tpr)
  auc3.append(roc_auc)
  proba3.append(model.predict_proba(cov_test))

(180, 32, 100)
(60, 32, 100)
(60, 32, 100)
(180, 4656)
(180,)
(60, 4656)
(60,)
(60, 4656)
(180, 96, 96)
(60, 96, 96)
(60, 96, 96)
[14:51:16] WARNING: ../src/learner.cc:516: 
Parameters: { probability } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


(180, 32, 100)
(60, 32, 100)
(60, 32, 100)
(180, 4656)
(180,)
(60, 4656)
(60,)
(60, 4656)
(180, 96, 96)
(60, 96, 96)
(60, 96, 96)
[14:52:06] WARNING: ../src/learner.cc:516: 
Parameters: { probability } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


(180, 32, 100)
(60, 32, 100)
(60, 32, 100)
(180, 4656)
(180,)
(60, 4656)
(60,)

In [ ]:
auc1=np.array(auc1)
auc2=np.array(auc2)
auc3=np.array(auc3)
proba1=np.array(proba1)
proba2=np.array(proba2)
proba3=np.array(proba3)
val_proba1=np.array(val_proba1)
val_proba2=np.array(val_proba2)
val_proba3=np.array(val_proba3)
score1=np.array(score1)
score2=np.array(score2)
score3=np.array(score3)
print(auc1.shape)
print(auc2.shape)
print(auc3.shape)
print(proba1.shape)
print(proba2.shape)
print(proba3.shape)
print(val_proba1.shape)
print(val_proba2.shape)
print(val_proba3.shape)
print(score1.shape)
print(score2.shape)
print(score3.shape)

(15,)
(15,)
(15,)
(15, 60, 2)
(15, 60, 2)
(15, 60, 2)
(15, 60, 2)
(15, 60, 2)
(15, 60, 2)
(15,)
(15,)
(15,)


In [ ]:
score1

array([0.41666667, 0.6       , 0.61666667, 0.51666667, 0.6       ,
       0.76666667, 0.35      , 0.76666667, 0.66666667, 0.41666667,
       0.45      , 0.48333333, 0.71666667, 0.45      , 0.48333333])

In [ ]:
auc1

array([0.48918269, 0.512     , 0.504     , 0.35403727, 0.61490683,
       0.46118012, 0.41614907, 0.60714286, 0.50721154, 0.544     ,
       0.558     , 0.49147727, 0.65625   , 0.37784091, 0.54403409])

In [ ]:
auc2

array([0.53004808, 0.534     , 0.428     , 0.58307453, 0.62732919,
       0.63354037, 0.44642857, 0.47670807, 0.41826923, 0.453     ,
       0.704     , 0.4140625 , 0.68039773, 0.32528409, 0.39417614])

In [ ]:
auc3

array([0.60456731, 0.439     , 0.531     , 0.47670807, 0.51552795,
       0.57531056, 0.58618012, 0.51009317, 0.44230769, 0.409     ,
       0.416     , 0.58806818, 0.65980114, 0.47727273, 0.52840909])

In [ ]:
proba4=[]
proba5=[]
val_proba4=[]
val_proba5=[]
auc=[]
aucc=[]
for i in range(15):
  p1=proba1[i,:,1]
  p2=proba2[i,:,1]
  p3=proba3[i,:,1]
  preds=(p1+p2+p3)/3
  preds2=np.maximum(p1,p2,p3)
  proba4.append(preds)
  proba5.append(preds2)
  p1=val_proba1[i,:,1]
  p2=val_proba2[i,:,1]
  p3=val_proba3[i,:,1]
  preds=(p1+p2+p3)/3
  preds2=np.maximum(p1,p2,p3)
  val_proba4.append(preds)
  val_proba5.append(preds2)
  fpr, tpr, threshold = metrics.roc_curve(validation_labels, preds)
  roc_auc = metrics.auc(fpr, tpr)
  auc.append(roc_auc)
  fpr, tpr, threshold = metrics.roc_curve(validation_labels, preds2)
  roc_auc = metrics.auc(fpr, tpr)
  aucc.append(roc_auc)
auc=np.array(auc)
aucc=np.array(aucc)
proba4=np.array(proba4)
proba5=np.array(proba5)
val_proba4=np.array(val_proba4)
val_proba5=np.array(val_proba5)

In [ ]:
auc

array([0.54829545, 0.31107955, 0.5       , 0.67755682, 0.31960227,
       0.41619318, 0.421875  , 0.45170455, 0.52130682, 0.48295455,
       0.45596591, 0.50852273, 0.6875    , 0.35085227, 0.45880682])

In [ ]:
aucc

array([0.53053977, 0.37073864, 0.49786932, 0.70596591, 0.35227273,
       0.44602273, 0.69886364, 0.37784091, 0.50710227, 0.42471591,
       0.36292614, 0.48650568, 0.70454545, 0.35653409, 0.47585227])

In [ ]:
sub=int(input("enter subject number:"))

In [ ]:
sub

In [ ]:
auc1[sub-1]

In [ ]:
auc2[sub-1]

In [ ]:
auc3[sub-1]

In [ ]:
auc[sub-1]

In [ ]:
aucc[sub-1]

In [ ]:
proba3[sub-1,:,1]

In [ ]:
for i in range(15):
  print(max(auc1[i],auc2[i],auc3[i],auc[i],aucc[i]))

In [ ]:
score1

array([0.41666667, 0.6       , 0.61666667, 0.51666667, 0.6       ,
       0.76666667, 0.35      , 0.76666667, 0.66666667, 0.41666667,
       0.45      , 0.48333333, 0.71666667, 0.45      , 0.48333333])